In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Input,Dense
from keras.utils import to_categorical
from keras.datasets import cifar10

In [24]:
num_classes=10
input_shape=(32, 32, 3)

## 数据预处理

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
# 数据增强
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_datagen.fit(x_train)

## 创建模型

In [26]:
def create_model():
    image_input=Input(shape=input_shape)
    conv1=Conv2D(32,kernel_size=(3,3),activation='relu',padding='same')(image_input)
    conv2=Conv2D(32,kernel_size=(3,3),activation='relu')(conv1)
    pool1=MaxPool2D(pool_size=(2, 2))(conv2)
    pool1=Dropout(0.25)(pool1)
    conv3=Conv2D(64, kernel_size=(3,3),activation='relu')(pool1)
    conv4=Conv2D(64, kernel_size=(3,3),activation='relu')(conv3)
    pool2=MaxPool2D(pool_size=(2, 2))(conv4)
    pool2=Dropout(0.25)(pool2)
    dense=Flatten()(pool2)
    dense=Dense(512,activation='relu')(dense)
    dense=Dropout(0.5)(dense)
    preds=Dense(10, activation='softmax')(dense)
    model=Model(image_input,preds)
    model.compile(optimizer='rmsprop',loss="categorical_crossentropy",metrics=["accuracy"])
    model.summary()
    return model
model=create_model()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 11, 11, 64)        3692

## 训练

In [27]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=10)

Epoch 1/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.6282 - accuracy: 0.4101
Epoch 2/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.3043 - accuracy: 0.5412
Epoch 3/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.1939 - accuracy: 0.5889
Epoch 4/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.1426 - accuracy: 0.6099
Epoch 5/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.1288 - accuracy: 0.6165
Epoch 6/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.1443 - accuracy: 0.6182
Epoch 7/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.1626 - accuracy: 0.6152
Epoch 8/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.1961 - accuracy: 0.6076
Epoch 9/10
1563/1562 [==============================] - 15s 9ms/step - loss: 1.2233 - accuracy: 0.5978
Epoch 10/10
1563/1562 [==============================] - 15s 10ms/step - 

## 预测

In [28]:
test_datagen = ImageDataGenerator(rescale=1./255)
prediction=model.predict_generator(test_datagen.flow(x_test, batch_size=1))

In [29]:
prediction

array([[9.58915725e-02, 2.48960897e-01, 1.79311857e-02, ...,
        6.63576042e-03, 4.54858810e-01, 1.39545649e-01],
       [8.52727983e-03, 7.83650130e-02, 8.82445544e-04, ...,
        1.83691317e-03, 2.23307274e-02, 8.83106828e-01],
       [3.32298204e-02, 2.32766028e-02, 9.27049071e-02, ...,
        1.27313018e-01, 7.14371800e-02, 9.19944793e-02],
       ...,
       [7.71535560e-05, 4.56664784e-05, 2.25497782e-02, ...,
        8.46468378e-04, 6.69107249e-04, 5.61139714e-05],
       [7.91490136e-04, 4.86954639e-04, 1.27374172e-01, ...,
        1.07564665e-01, 2.23355644e-04, 1.64540869e-03],
       [3.39485168e-01, 8.31511840e-02, 5.49951307e-02, ...,
        7.97751397e-02, 2.35293686e-01, 1.08191028e-01]], dtype=float32)